# Example Brick Model for process
Example brick model will be 2 AHUs, 60 terminal units.
Half are for perimeter zones vavs with reheat, half are for core zones and are cooling only 

10 core zone terminal units are for mechanical rooms, networking closets, and electrical rooms that must be served 24/7

In [2]:
from rdflib import Namespace, URIRef, RDFS, Graph, Literal
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Model, Library
from buildingmotif.namespaces import BMOTIF, BRICK, A, RDF
from buildingmotif.model_builder import TemplateBuilderContext

In [3]:
BLDG = Namespace("urn:example#")
bm = BuildingMOTIF("sqlite://", "topquadrant")
bldg = Model.create("urn:example#")
bldg.graph.bind('',BLDG)
# brick = Library.load(ontology_graph="Brick.ttl")

In [4]:
s223_tmpl = Library.load(directory='s223-templates')

In [6]:
# bldg_config
# assuming cooling+only and reheat vavs split evenly. Computer spaces are all in the cooling only vavs.
# [ahus, cooling_only_vavs_per_ahu, reheat_vavs_per_ahu]
# Could add amt computer spaces_per_ahu to building if it serves example later

# the context helps us build up our model
ctx = TemplateBuilderContext(BLDG)
# ctx.add_templates_from_library(brick)
ctx.add_templates_from_library(s223_tmpl)

In [9]:
# TODO: MESSED UP CODE FLOW, NOW NEED TO RUN AGAIN FROM HERE
def get_wrapper_dict(ctx):
    wrapper_dict = {}
    for wrapper in ctx.wrappers:
        if wrapper.template.name not in wrapper_dict.keys():
            wrapper_dict[wrapper.template.name] = [wrapper]
        else:
            wrapper_dict[wrapper.template.name].append(wrapper)
    return wrapper_dict

In [10]:
wrapper_dict = get_wrapper_dict(ctx)
template_types = wrapper_dict.keys()
print(template_types)

dict_keys([])


In [13]:
g = Graph()
g.bind('',BLDG)

bldg_config = {'ahus': 1, 'cv': 1, 'hv': 1}
bldg_ctx_dict = {}
for i in range(bldg_config['ahus']):
    template_name = 'multiple-zone-ahu'
    ahu_name = f'ahu_{i}'
    ahu = ctx[template_name](name=ahu_name)
    ahu.bindings = {k: BLDG[f"{template_name}_{k}_{i}"] for k in ahu.parameters}
    # display(ahu.bindings)
    # also adding to condensed representation 
    for k,v in ahu.bindings.items():
        g.add((BLDG[f"{template_name}-{k.replace('name-','')}"], RDF[f"_{i+1}"], v))
    for cv in range(bldg_config['cv']):
        template_name = 'vav-cooling-only'
        cv_name = f'cvav_{i}_{cv}'
        cvav = ctx[template_name](name=cv_name)
        cvav.bindings = {k: BLDG[f"{template_name}_{k}_{i}_{cv}"] for k in cvav.parameters}
        # ctx['feeds'](name = ahu['name'], target = cvav['name'])
        for k,v in cvav.bindings.items():
            g.add((BLDG[f"{template_name}-{k.replace('name-','')}"], RDF[f"_{cv+1}"], v))
        # print(cvav.bindings)
    for hv in range(bldg_config['hv']):
        template_name = 'vav-reheat'
        hv_name = f'hvav_{i}_{hv}'
        hvav = ctx[template_name](name=hv_name)
        hvav.bindings = {k: BLDG[f"{template_name}_{k}_{i}_{hv}"] for k in hvav.parameters}
        # ctx['feeds'](name = ahu['name'], target = hvav['name'])
        for k,v in hvav.bindings.items():
            g.add((BLDG[f"{template_name}-{k.replace('name-','')}"], RDF[f"_{hv+1}"], v))

wrapper_dict = get_wrapper_dict(ctx)
template_types = wrapper_dict.keys()

skip_templates = ['feeds']
for tp in template_types:
    if tp in skip_templates:
        continue
    for wrapper in wrapper_dict[tp]:
        params = wrapper.parameters
        eval_dict = {}
        for param in params:
            # if param not in TEMPLATE_PARAMS_COMPILED[tp].keys():
            #     print(param)
            # g.add((BLDG[f"{tp}-{param.replace('name-','')}"], A, BRICK[TEMPLATE_PARAMS_COMPILED[tp][param]]))
            name = f"{tp}-{param.replace('name-','')}"
            g.add((BLDG[name], A, RDF.Seq))
            eval_dict[param] = BLDG[name]
        template_graph = wrapper.template.evaluate(eval_dict)
        if not isinstance(template_graph, Graph):
            print('not a graph, cant add')
        g = g + template_graph
    # adding feeds manually 
    # g.add((BLDG['multiple-zone-ahu-name'], BRICK.feeds, BLDG['vav-cooling-only-name']))
    # g.add((BLDG['multiple-zone-ahu-name'], BRICK.feeds, BLDG['vav-reheat-name']))

In [15]:
cvav.bindings

{'discharge-air-flow-sensor': rdflib.term.URIRef('urn:example#vav-cooling-only_discharge-air-flow-sensor_0_0'),
 'out': rdflib.term.URIRef('urn:example#vav-cooling-only_out_0_0'),
 'discharge-air-flow-setpoint': rdflib.term.URIRef('urn:example#vav-cooling-only_discharge-air-flow-setpoint_0_0'),
 'damper-actuator': rdflib.term.URIRef('urn:example#vav-cooling-only_damper-actuator_0_0'),
 'duct-before-damper': rdflib.term.URIRef('urn:example#vav-cooling-only_duct-before-damper_0_0'),
 'damper': rdflib.term.URIRef('urn:example#vav-cooling-only_damper_0_0'),
 'damper-position': rdflib.term.URIRef('urn:example#vav-cooling-only_damper-position_0_0'),
 'out-connection': rdflib.term.URIRef('urn:example#vav-cooling-only_out-connection_0_0'),
 'discharge-air-flow': rdflib.term.URIRef('urn:example#vav-cooling-only_discharge-air-flow_0_0'),
 'damper-in-mapsto': rdflib.term.URIRef('urn:example#vav-cooling-only_damper-in-mapsto_0_0'),
 'damper-out': rdflib.term.URIRef('urn:example#vav-cooling-only_da

In [19]:
g.serialize('s223-example-groups.ttl', format = 'ttl')

<Graph identifier=Nb53bc45c8adb4488bdc37fb2ac7836f6 (<class 'rdflib.graph.Graph'>)>

In [20]:
g_no_data = g.query("""
            CONSTRUCT {
                   ?s ?p ?o .
                    ?s a ?type .
                    ?o a ?otype .
            }
            WHERE {
            ?s ?p ?o .
            ?s a ?type .
            ?o a ?otype .
            FILTER EXISTS {
                    ?o a rdf:Seq .
                }
            FILTER (?type != rdf:Seq) .
            
            FILTER (?otype != rdf:Seq) .
            }
                   """).graph

def get_label(s):
    name= str(s).rsplit('-')[-1] if str(s).rsplit('-')[-1] != 'name' else str(s).rsplit('-')[-2]
    return Literal(name)
for s,p,o in g_no_data.triples((None,None,None)):    
    if p == A:
        continue               
    if p == BRICK.hasPoint:
        g_no_data.add((o, A, BRICK.Point))
    g_no_data.add((s, RDFS.label, get_label(s)))
    g_no_data.add((o, RDFS.label, get_label(o)))

g_no_data.bind('bldg',BLDG)

In [21]:
g_no_data.serialize("s223-groups-nd.ttl", format = 'ttl')

<Graph identifier=N676e3f455f054499b8c5399661cac2a2 (<class 'rdflib.graph.Graph'>)>

In [22]:
bldg.add_graph(ctx.compile())
bldg.graph.serialize('s223-example.ttl', format = 'ttl')

<Graph identifier=6390405b-3b01-4304-9c27-2961ff0bef99 (<class 'rdflib.graph.Graph'>)>